In [ ]:
import os
import json
import logging
import sys
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import det_curve
import numpy as np
import numpy as np
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report
import seaborn as sns


In [ ]:

def read_json(path):
    """ Read a json file from the given path."""
    with open(path, 'r') as f:
        data = json.load(f)
    return data

def write_json(path, data):
    """ Write a json file to the given path."""
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))

    with open(path, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
def compute_acc(prediction, ground, gt_relations, task_id):
  """
  Compute accuracy and handle multiclass data for DET curve.

  Args:
      prediction: Predicted labels.
      ground: True labels.

  Returns:
      acc: Accuracy score.
  """
  # Get unique labels from both prediction and ground truth to ensure all classes are included
  labels = list(set([item['relation'] for item in gt_relations]))

  labels = sorted(labels)
  labels = [label for label in labels if label !='']

  # acc = accuracy_score(ground, prediction)

  prediction = [pred if pred != '' else "undefined" for pred in prediction]
  ground = [g if g != '' else "undefined" for g in ground]
  # prediction = [pred   for pred in prediction if pred != '']
  # ground = [g  for g in ground if g != '']
  # Create a confusion matrix
  conf_matrix = confusion_matrix(ground, prediction, labels=labels)

  x_labels = labels
  y_labels = labels
  # x_labels.append("undefined")

  # Create a heatmap
  plt.figure(figsize=(14, 12))  # Slightly larger for better readability
  sns.heatmap(conf_matrix,
              annot=True,
              fmt='d',
              cmap='BuPu',
              xticklabels=x_labels,
              yticklabels=labels,
              annot_kws={"size": 24})  # Increased font size for annotations
  plt.xlabel('Predicted', fontsize=24, fontweight='bold')   # Larger font for axis labels
  plt.ylabel('True', fontsize=24, fontweight='bold')
  plt.title('Confusion Matrix Heatmap - Flan T5 \nOver Incremental Learning', fontsize=28, fontweight='bold')   # More prominent title
  plt.xticks(fontsize=24, rotation=30, fontweight='bold')  # Rotate x-axis tick labels by 45 degrees
  plt.yticks(fontsize=24, rotation=30, fontweight='bold')  # Rotate y-axis tick labels by 45 degrees


  # Adjust layout to prevent overlap
  plt.tight_layout()

  # Save as a high-resolution image
  plt.savefig(f'./confusion_matrix_t5/t5_task1_over_til_confusion_matrix_task_{task_id}.pdf', dpi=300)

  plt.show()




  return 0

**FewRel**

In [ ]:
def get_gt(gt_base, task_id, run_id):
  gts=[]
  print("test")
  print(gt_base)
  for i in range(1, task_id+1):
    gt_path = f'{gt_base}/run_{run_id}/task{i}/test_1.json'
    gt = read_json(gt_path)
    gts.extend([item['relation'] for item in gt])
  return gts

In [ ]:
def get_list_acc(gt_path_base, experiment_result_path, task_1_path, model="t5"):
  results = []
  for run_id   in range(1, 6):
    gt_path = f'{gt_path_base}/run_{run_id}/task1/test_1.json'
    gt = read_json(gt_path)
    gt_relations = [item['relation'] for item in gt]
    size_task1 = len(gt_relations)
    task_1 = read_json(f'{experiment_result_path}/KMmeans_CRE_fewrel_{run_id}/task_task1_current_task_pred.json')
    if model=='t5':
      results.append({"run":run_id, "task":1, "acc":task_1[0]['acc']})
    else:
      task_1_pred = [item['predict'] for item in task_1]
      # Filter out None values from both lists before calculating accuracy, ensuring the lists remain aligned
      # Zip the two lists to iterate through them in parallel
      filtered_data = [(p, g) for p, g in zip(task_1_pred, gt_relations) if p is not None and g is not None]
      # If filtered_data is empty, set task_1_pred_filtered and gt_relations_filtered to empty lists to avoid errors
      if not filtered_data:
          task_1_pred_filtered = []
          gt_relations_filtered = []
      else:
          # Unzip the filtered data back into separate lists
          task_1_pred_filtered, gt_relations_filtered = zip(*filtered_data)
      acc = compute_acc(task_1_pred, gt_relations, gt, 1 )
      results.append({"run":run_id, "task":1, "acc":acc})
    for task_id in range(2, 11):
      print(task_id)
      pred_path = f'{task_1_path}/KMmeans_CRE_fewrel_{run_id}/task_{task_id}_seen_task.json'
      pred_task_1 = read_json(pred_path)[:len(gt)]
      pred_relations_task_1 = [item['predict'] for item in pred_task_1]
      # Filter out None values from both lists before calculating accuracy, ensuring the lists remain aligned
      # Zip the two lists to iterate through them in parallel
      gt_relations = get_gt(gt_path_base, task_id, run_id)[:len(gt)]
      filtered_data = [(p, g) for p, g in zip(pred_relations_task_1, gt_relations) if p is not None and g is not None]
      # If filtered_data is empty, set pred_relations_task_1_filtered and gt_relations_filtered to empty lists to avoid errors
      if not filtered_data:
          pred_relations_task_1_filtered = []
          gt_relations_filtered = []
      else:
          # Unzip the filtered data back into separate lists
          pred_relations_task_1_filtered, gt_relations_filtered = zip(*filtered_data)
      task_1_acc = compute_acc(pred_relations_task_1_filtered, gt_relations_filtered, gt, task_id)
      results.append({"run":run_id, "task":task_id, "acc":task_1_acc})
  return results

In [ ]:
t5_results = get_list_acc('t5_format/test', 't5', 't5', "t5_")

In [ ]:

t5_df = pd.DataFrame(t5_results)
mean_acc_t5 = t5_df.groupby('task').mean()
std_acc_t5 = t5_df.groupby('task').std()

In [ ]:
mistral_results = get_list_acc('/content/llama_format_data/test', '/content/mistral_fewrel_clean/m_10', '/content/mistral_fewrel_clean/m_10', 'mistral')


In [ ]:

mean_mistral_tacred = pd.DataFrame(mistral_results).groupby('task').mean()['acc']
std_mistral_tacred = pd.DataFrame(mistral_results).groupby('task').std()['acc']

In [ ]:
llama_results = get_list_acc('/content/llama_format_data/test', '/content/llama_seen_clean_mist_code', '/content/llama_seen_clean_mist_code', 'llama')

In [ ]:
mean_llama_tacred = pd.DataFrame(llama_results).groupby('task').mean()['acc']
std_llama_tacred = pd.DataFrame(llama_results).groupby('task').std()['acc']

In [ ]:


# Set up figure size
plt.figure(figsize=(10, 8))

# Generate example data
x = [f'{i}' for i in range(1, 11)]  # X values
mean_t5 = mean_acc_t5['acc']  # Line values for Flan T5 Base
std_t5 = std_acc_t5['acc']  # Standard deviation for Flan T5 Base

# Scale data to percentages


# Plot the Flan T5 Base line
plt.plot(x, mean_t5, label='Flan T5 Base', color='purple',  marker='o')

# Plot other models
plt.plot(x, mean_mistral_tacred, label='Mistral-7b-Instruct-v2.0', color='orange',  marker='o')
plt.plot(x, mean_llama_tacred, label='Llama-2-7b-chat-hf', color='green',  marker='o')

# Optionally add the standard deviation as shaded areas (commented out for now)
plt.fill_between(x, np.array(mean_t5) - np.array(std_t5), np.array(mean_t5) + np.array(std_t5),
                 color='purple', alpha=0.1)
plt.fill_between(x, np.array(mean_mistral_tacred) - np.array(std_mistral_tacred),
                 np.array(mean_mistral_tacred) + np.array(std_mistral_tacred),
                 color='orange', alpha=0.1)
plt.fill_between(x, np.array(mean_llama_tacred) - np.array(std_llama_tacred),
                 np.array(mean_llama_tacred) + np.array(std_llama_tacred),
                 color='green', alpha=0.05)

# Customize the plot
plt.title("Task 1 Test Accuracy \n across Incremental Learning", fontsize=26, fontweight='bold')
plt.xlabel("Base Training Task Index", fontsize=26, fontweight='bold')
plt.ylabel("Mean Accuracy (%)", fontsize=26, fontweight='bold')
plt.xticks(fontsize=20, fontweight='bold')  # Rotate x-axis labels and increase font size
plt.yticks(fontsize=20, fontweight='bold')  # Increase font size of y-axis labels
plt.legend(fontsize=20)
plt.grid(True)

# Improve layout and save the figure
plt.tight_layout()
plt.savefig("task1_accuracy_fewrel.pdf", dpi=300)
plt.show()
